In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

file_path = "D:\\MRI\\OAS2_0002_MR1\\1\\123127134\\mpr-1.nifti.img"

"D:\MRI - Copy\OAS2_0002_MR1\1\123127134\mpr-1.nifti.img"# Load the NIfTI image
image = nib.load(file_path)
                                
            
# Get the image data
image_data = image.get_fdata()
# Display the image
plt.imshow(image_data[:, :, 70], cmap=None)  # Display a middle slice (in this case, slice 64)
plt.axis('off')
plt.show()



In [ ]:
import ants

image_path = "D:\\biasmpr-1.nifti.img"

image = ants.image_read(image_path)

resampled_image = ants.resample_image(image, (1, 1, 1))

resampled_image_path = "D:\\resampledmpr-1.nifti.img"
ants.image_write(resampled_image, resampled_image_path)

In [ ]:
import ants

image_path = "D:\\resampledmpr-1.nifti.img"

image = ants.image_read(image_path)

denoised_image = ants.denoise_image(image, noise_model='Gaussian')

denoised_image_path = "D:\\deionisedmpr-1.nifti.img"
ants.image_write(denoised_image, denoised_image_path)

In [ ]:
import ants

image_path = "D:\\deionisedmpr-1.nifti.img"

image = ants.image_read(image_path)

normalized_image = ants.iMath(image, 'Normalize')

normalized_image_path = "D:\\normaliseddmpr-1.nifti.img"
ants.image_write(normalized_image, normalized_image_path)

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
image_path = "D:\\MRI - Copy\\OAS2_0001_MR2\\1\\123127131\\mpr-1.nifti.img"
image = nib.load(image_path)
image_data = image.get_fdata()
# Display a single slice
slice_index = 50  # Specify the desired slice index
plt.imshow(image_data[:, :, slice_index], cmap=None)
plt.title("MRI Slice")
plt.axis("off")
plt.show()
from skimage.draw import polygon

# Specify the coordinates of the ROI
x1, y1 = 10, 60  # Top-left corner
x2, y2 = 210, 115  # Top-right corner
x3, y3 = 180, 180  # Bottom-right corner
x4, y4 = 40, 180  # Bottom-left corner
x5, y5 = 20, 145
# Create a binary mask for the rectangular ROI
roi_mask = np.zeros_like(image_data)
rr, cc = polygon([x1, x2, x3, x4,x5], [y1, y2, y3, y4,y5])
roi_mask[rr, cc, slice_index] = 1

# Apply the ROI mask to the MRI slice
roi_slice = image_data[:, :, slice_index] * roi_mask[:, :, slice_index]

# Display the ROI slice
plt.imshow(roi_slice, cmap=None)
plt.title("ROI Slice")
plt.axis("off")
plt.show()

In [ ]:
import nibabel as nib

# Load the .nifti file
nifti_img = nib.load("D:\\MRI\\OAS2_0001_MR2\\1\\123127131\\mpr-1.nifti.img")

# Access the image data
image_data = nifti_img.get_fdata()

# Print the image shape
print('Image shape:', image_data.shape)


In [ ]:
# Access the voxel dimensions
voxel_dims = nifti_img.header['pixdim'][1:4]
print('Voxel dimensions:', voxel_dims)

# Access the image intensity values
intensity_values = image_data.flatten()
print('Min intensity:', intensity_values.min())
print('Max intensity:', intensity_values.max())


In [ ]:
import numpy as np
from nilearn import image

# Resample the image to isotropic voxel dimensions of 1mm
resampled_img = image.resample_img(nifti_img, target_affine=np.array([(1, 0, 0, 0),
                                                                      (0, 1, 0, 0),
                                                                      (0, 0, 1, 0),
                                                                      (0, 0, 0, 1)]),
                                   target_shape=(256, 256, 256))


In [ ]:
# Normalize the intensity values to the range [0, 1]
normalized_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))


In [ ]:
from scipy.ndimage import gaussian_filter

# Apply Gaussian filtering with a sigma value of 1
denoised_data = gaussian_filter(image_data, sigma=1)


In [ ]:
import matplotlib.pyplot as plt

# Display a slice of the image
slice_index = 100
plt.imshow(image_data[:, :, slice_index], cmap=None)
plt.axis('off')
plt.show()


In [ ]:
from nipype.interfaces.ants.segmentation import N4BiasFieldCorrection
from dipy.segment.tissue import TissueClassifierHMRF
from dipy.io.image import load_nifti_data
from __future__ import print_function
from deepbrain import Extractor
import matplotlib.pyplot as plt
import SimpleITK as sitk
import nibabel as nib

import time
import os
import sys
import subprocess
import shutil
import numpy as np
  
class Preprocess:

  def __init__(self):
    
    
    print('Start preprocessing....')

  def strip_the_skull(self,src_path,dst_path):
  
    img = nib.load(src_path)
    affine = img.affine
    img = img.get_fdata()
    ext = Extractor()
    prob = ext.run(img)
    mask = prob > 0.5
    brain = img[:]
    brain[~mask] = 0
    brain = nib.Nifti1Image(brain, affine)
    nib.save(brain, dst_path)

    
  def get_noiseless_image(self,src_path,dst_path):
  
    try:
      inputImage = sitk.ReadImage(src_path)
      img_data=sitk.Cast(inputImage,sitk.sitkFloat32) 
      img_mask=sitk.BinaryNot(sitk.BinaryThreshold(img_data, 0, 0))
      corrected_img = sitk.N4BiasFieldCorrection(inputImage, img_mask)
      sitk.WriteImage(corrected_img, dst_path)
      print('Succesfully performed N4 Bias Correction')
    
    except RuntimeError:
      #Runs if any stmts of Bias Correction fails
      print('Failed on :' + src_path)  
  
  def do_segmentation(self,src_path,dst_path):
   
    #To get only the image data array
    t1 = load_nifti_data(src_path)
    print('t1.shape (%d, %d, %d)' % t1.shape)
    #To load the entire nifti file
    t2 = nib.load(src_path)
    nclass = 3
    beta = 0.1
    t0 = time.time()
    hmrf = TissueClassifierHMRF()
    #Perform segmentation
    initial_segmentation, final_segmentation, PVE = hmrf.classify(t1, nclass, beta)
    t1 = time.time()
    total_time = t1-t0
    print('Total time:' + str(total_time))
    print(final_segmentation.shape)
    brain = nib.Nifti1Image(final_segmentation,t2.affine)
    print('Segmentation performed successfully')
    nib.save(brain, dst_path)


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

# Define the root directory containing the image folders
root_dir = 'D:\\MRI - copy\\'
input_image = Preprocess()
# Iterate over the folders
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    
    # Check if the item is a directory
    if os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for folder1 in os.listdir(folder_path):
            folder1_path = os.path.join(folder_path, folder1)
            path_to_store_stripped_skull=os.path.join(folder1_path,"sssmpr.nifti.img")
            path_to_store_segmented_img=os.path.join(folder1_path,"segmentmpr.nifti.img")
            
            if os.path.isdir(folder1_path):
                # Iterate over the files in the folder
                for folder2 in os.listdir(folder1_path):
                    folder2_path = os.path.join(folder1_path, folder2)
                    
                    if os.path.isdir(folder2_path):
                        
                        # Iterate over the files in the folder
                        for folder3 in os.listdir(folder2_path):
                            folder3_path = os.path.join(folder2_path, folder3)
                            
                            
                            # Check if the item is a file
                            if folder3_path.endswith('mpr-1.nifti.img'):
                                input_image.strip_the_skull(folder3_path,path_to_store_stripped_skull)
                                input_image.do_segmentation(path_to_store_stripped_skull,path_to_store_segmented_img)

                                # Get the image data
                                #image_data = image.get_fdata()

                                # Display the image
                                #plt.imshow(image_data[:, :, 64], cmap=None)  # Display a middle slice (in this case, slice 64)
                                #plt.axis('off')
                                #plt.show()



In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
df=pd.DataFrame(columns=['contrast','dissimilarity','homogeneity','energy','correlation'])
df

root_dir = 'D:\\MRI - copy\\'

for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    
    # Check if the item is a directory
    if os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for folder1 in os.listdir(folder_path):
            folder1_path = os.path.join(folder_path, folder1)
            
            if os.path.isdir(folder1_path):
                        
                # Iterate over the files in the folder
                for folder2 in os.listdir(folder1_path):
                    folder2_path = os.path.join(folder1_path, folder2)
                            
                            
                        # Check if the item is a file
                    if folder2_path.endswith('segmentmpr.nifti.img'):
                        print(folder2_path)
                        image = nib.load(folder2_path)
                        segmented_image_data = image.get_fdata()

                        # Check the dimensions of the segmented image
                        num_slices = segmented_image_data.shape[2]
                        print("Number of slices:", num_slices)

                        # Select a slice for processing (e.g., slice_index = 0)
                        slice_index = 0
                        slice_image = segmented_image_data[:, :, slice_index]

                        gray_image = slice_image.squeeze()
                        
                        gray_image = np.uint8(gray_image)

                        # Define the distances and angles for GLCM calculation
                        distances = [1]  # Distance between pixels
                        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles for texture analysis

                        # Compute the GLCM matrix
                        glcm = greycomatrix(gray_image, distances, angles, symmetric=True, normed=True)

                        # Calculate texture features from the GLCM matrix
                        contrast = greycoprops(glcm, 'contrast')
                        dissimilarity = greycoprops(glcm, 'dissimilarity')
                        homogeneity = greycoprops(glcm, 'homogeneity')
                        energy = greycoprops(glcm, 'energy')
                        correlation = greycoprops(glcm, 'correlation')
                        
                        print("Contrast:", contrast)
                        print("Dissimilarity:", dissimilarity)
                        print("Homogeneity:", homogeneity)
                        print("Energy:", energy)
                        print("Correlation:", correlation)
                        
                        df.loc[len(df.index)] = [contrast,dissimilarity,homogeneity,energy,correlation]
                            
df
            

In [ ]:
df1=pd.read_excel("D:\\Datasets\\oasis_longitudinal_demographics.xlsx")
df1=df1[['MRI ID','Group']]
df1

In [ ]:
df2 = df1.join(df)
df2=df2.set_index('MRI ID')
df2= df2[df2['Group'] != 'Converted']
df2

In [ ]:
for column in df2.drop('Group',axis=1).columns:
    df2[column] = df2[column].apply(lambda x: np.array(x).flatten())
df2

In [ ]:
for column in df2.drop('Group', axis=1).columns:
    for i in range(len(df[column][0])):
        df2[f'{column}_element{i+1}'] = df2[column].apply(lambda x: x[i])
        df2[f'{column}_element{i+2}'] = df2[column].apply(lambda x: x[i+1])
        df2[f'{column}_element{i+3}'] = df2[column].apply(lambda x: x[i+2])
        df2[f'{column}_element{i+4}'] = df2[column].apply(lambda x: x[i+3])
        
# Drop the original feature columns if desired
df2 = df2.drop(['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation'], axis=1)
df2

In [ ]:
df2['contrast'] = df2[['contrast_element1','contrast_element2','contrast_element3','contrast_element4']].mean(axis=1)
df2['dissimilarity'] = df2[['dissimilarity_element1','dissimilarity_element2','dissimilarity_element3','dissimilarity_element4']].mean(axis=1)
df2['homogeneity'] = df2[['homogeneity_element1','homogeneity_element2','homogeneity_element3','homogeneity_element4']].mean(axis=1)
df2['energy'] = df2[['energy_element1','energy_element2','energy_element3','energy_element4']].mean(axis=1)
df2['correlation'] = df2[['correlation_element1','correlation_element2','correlation_element3','correlation_element4']].mean(axis=1)


In [ ]:
df2=df2[['Group','contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]
df2

In [ ]:
df2.drop(['dissimilarity','contrast'],axis=1).corr()

In [ ]:
df

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Step 1: Prepare the Data
X = df2.drop(['Group','contrast'],axis=1)  # Numerical features
y = df2['Group']  # Textual class label

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Step 2: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3,random_state=42)

# Encode the target variable


# Step 3: Preprocess the Data (if needed)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Build the Neural Network Model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

# Step 5: Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 6: Train the Model
history = model.fit(X_train_scaled, y_train, epochs=15, batch_size=8, verbose=1, validation_data=(X_test_scaled, y_test))

# Step 7: Make Predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.where(y_pred_prob >= 0.5, 1, 0)

# Step 8: Evaluate the Model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



y_pred_prob = model.predict(X_train_scaled)
y_pred = np.where(y_pred_prob >= 0.5, 1, 0)

print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))










import matplotlib.pyplot as plt


# Access the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training and validation loss
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.plot(epochs, train_acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
import visualkeras
visualkeras.layered_view(model, legend=True) # without custom font
from PIL import ImageFont
font = ImageFont.truetype("arial.ttf", 12)
visualkeras.layered_view(model, legend=True, font=font) # selected font

In [ ]:
model.summary()